# Find cluster related terms using LIME

## Load the data

In [1]:
root_dir = '../..'
data_dir = 'data'
corpus_dir = 'corpus'
src_dir = 'src'

In [2]:
import os 
import sys

In [3]:
sys.path.append(os.path.join(root_dir, src_dir))

In [4]:
dataset_name = 'alaska'

In [5]:
corpus_filename = f'{dataset_name}_corpus.json'
corpus_filepath = os.path.join(root_dir, data_dir, corpus_dir, corpus_filename)

In [6]:
chunks_filename = f'{dataset_name}_chunks.json'
chunks_filepath = os.path.join(root_dir, data_dir, corpus_dir, chunks_filename)

In [7]:
from training import TrainingCorpus

In [8]:
corpus = TrainingCorpus()
corpus.load(corpus_filepath)
corpus.load_chunks(chunks_filepath)

---

## Load the model

Check if GPU is available

In [9]:
import torch
print(torch.cuda.is_available())

True


In [10]:
from model import BertModel

Using TensorFlow backend.


In [11]:
model_dir = f'models/{dataset_name}_bert_test'
model_dir_path = os.path.join(root_dir, data_dir, model_dir)

In [12]:
model = BertModel(model_dir_path, batch_size=512, use_cuda=True, from_tf=False)

---

## Find relevant terms for each cluster label using LIME

### Instantiate TermFinder

In [13]:
from termfinder import LimeTermFinder

In [14]:
term_finder = LimeTermFinder(model, corpus, min_fts=5, max_fts=10)

### Retrieve predicted labels for each instance in the corpus

In [15]:
import numpy as np

In [16]:
label_to_data_idx_dict = model.label_to_data_idx(corpus)

In [17]:
len(label_to_data_idx_dict)

20

### Retrieve relevant terms using LimeTermFinder

In [18]:
from tqdm.notebook import tqdm

In [19]:
df_data = []

In [20]:
%%time
for label_idx, data_idxs in tqdm(label_to_data_idx_dict.items()):
    for data_idx in tqdm(data_idxs, desc=f'Relevant terms for entity {label_idx}', leave=False):
        relevant_terms = term_finder.get_relevant_terms(data_idx, label_idx)

        if relevant_terms:
            for term, weight in relevant_terms.items():
                dict_entry = {'label': corpus.labels[label_idx], 
                              'term': term, 
                              'weight': weight, 
                              'data_id': corpus.docs[data_idx]}
                df_data.append(dict_entry)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




CPU times: user 8h 8min 35s, sys: 1h 23min 24s, total: 9h 32min
Wall time: 1h 57min 43s


---